# Pandas.DataFrame.resample() Guide
by Jeremy Chow

9/10/2019

Notebook made using Pandas version 0.24.2

In [135]:
import pandas as pd

# Resampling Example Rules

### Downsampling Minutes to Every Other Minute

In [140]:
def new_dataset(rule,num_points = 9):
    '''
    Generate a new timeseries dataset using intervals in rule string
    and number of points num_points
    
    Returns a series object
    '''
    ind = pd.date_range('1/1/2000', periods=num_points, freq=rule)
    return pd.Series(list(map(lambda x: x*2,range(num_points))), index=ind)

In [144]:
minutes = new_dataset('min')
minutes

2000-01-01 00:00:00     0
2000-01-01 00:01:00     2
2000-01-01 00:02:00     4
2000-01-01 00:03:00     6
2000-01-01 00:04:00     8
2000-01-01 00:05:00    10
2000-01-01 00:06:00    12
2000-01-01 00:07:00    14
2000-01-01 00:08:00    16
Freq: T, dtype: int64

In [145]:
minutes.resample('2min',axis=0).sum()

2000-01-01 00:00:00     2
2000-01-01 00:02:00    10
2000-01-01 00:04:00    18
2000-01-01 00:06:00    26
2000-01-01 00:08:00    16
Freq: 2T, dtype: int64

### Upsampling Months to Bimonthly

In [146]:
months = new_dataset('M',5)
months

2000-01-31    0
2000-02-29    2
2000-03-31    4
2000-04-30    6
2000-05-31    8
Freq: M, dtype: int64

In [147]:
# ffill is Forward Fill, takes value of previous existing months point.
print('Forward Fill')
print(months.resample('SMS').ffill().tail())

# Alternatie to ffill is bfill (backward fill) that takes value of next existing months point
print('Backward Fill')
print(months.resample('SMS').bfill().tail())

Forward Fill
2000-03-15    2.0
2000-04-01    4.0
2000-04-15    4.0
2000-05-01    6.0
2000-05-15    6.0
Freq: SMS-15, dtype: float64
Backward Fill
2000-03-15    4
2000-04-01    6
2000-04-15    6
2000-05-01    8
2000-05-15    8
Freq: SMS-15, dtype: int64


### Upsample with NaNs Using asfreq()

In [148]:
minutes

2000-01-01 00:00:00     0
2000-01-01 00:01:00     2
2000-01-01 00:02:00     4
2000-01-01 00:03:00     6
2000-01-01 00:04:00     8
2000-01-01 00:05:00    10
2000-01-01 00:06:00    12
2000-01-01 00:07:00    14
2000-01-01 00:08:00    16
Freq: T, dtype: int64

In [149]:
minutes.resample('.5min').asfreq()

2000-01-01 00:00:00     0.0
2000-01-01 00:00:30     NaN
2000-01-01 00:01:00     2.0
2000-01-01 00:01:30     NaN
2000-01-01 00:02:00     4.0
2000-01-01 00:02:30     NaN
2000-01-01 00:03:00     6.0
2000-01-01 00:03:30     NaN
2000-01-01 00:04:00     8.0
2000-01-01 00:04:30     NaN
2000-01-01 00:05:00    10.0
2000-01-01 00:05:30     NaN
2000-01-01 00:06:00    12.0
2000-01-01 00:06:30     NaN
2000-01-01 00:07:00    14.0
2000-01-01 00:07:30     NaN
2000-01-01 00:08:00    16.0
Freq: 30S, dtype: float64

### Example of Complex Rule Strings

In [150]:
months

2000-01-31    0
2000-02-29    2
2000-03-31    4
2000-04-30    6
2000-05-31    8
Freq: M, dtype: int64

In [151]:
months.resample('3D7H30min180S',kind='timestamp').sum().head(10)

2000-01-31 00:00:00    0
2000-02-03 07:33:00    0
2000-02-06 15:06:00    0
2000-02-09 22:39:00    0
2000-02-13 06:12:00    0
2000-02-16 13:45:00    0
2000-02-19 21:18:00    0
2000-02-23 04:51:00    0
2000-02-26 12:24:00    2
2000-02-29 19:57:00    0
Freq: 4773T, dtype: int64

# Arguments


### Closed 
{'right', 'left'}, default None

As per the documentation:
>Which side of bin interval is closed. The default is 'left'
    for all frequency offsets except for 'M', 'A', 'Q', 'BM',
    'BA', 'BQ', and 'W' which all have a default of 'right'.
    
The closed argument tells which side is included, 'closed' being the included side in the calculation for each time interval. You can see how it behaves here:

In [162]:
minutes

2000-01-01 00:00:00     0
2000-01-01 00:01:00     2
2000-01-01 00:02:00     4
2000-01-01 00:03:00     6
2000-01-01 00:04:00     8
2000-01-01 00:05:00    10
2000-01-01 00:06:00    12
2000-01-01 00:07:00    14
2000-01-01 00:08:00    16
Freq: T, dtype: int64

In [177]:
# The default is closed='left'
df=pd.DataFrame()
df['left'] = minutes.resample('2min').sum()
df['right'] = minutes.resample('2min',closed='right').sum()
df

,left,right
2000-01-01 00:00:00,2,6.0
2000-01-01 00:02:00,10,14.0
2000-01-01 00:04:00,18,22.0
2000-01-01 00:06:00,26,30.0
2000-01-01 00:08:00,16,NaN


### Label
{'right', 'left'}, default None

Documentation
>Which bin edge label to label bucket with. The default is 'left'
    for all frequency offsets except for 'M', 'A', 'Q', 'BM',
    'BA', 'BQ', and 'W' which all have a default of 'right'.

This argument does not change the underlying calculation, it just relables the output based on the desired edge once the aggregation is performed.

In [186]:
df=pd.DataFrame()
# Label default is left
df['left'] = minutes.resample('2min').sum()
df['right'] = minutes.resample('2min',label='right').sum()
df

,left,right
2000-01-01 00:00:00,2,NaN
2000-01-01 00:02:00,10,2.0
2000-01-01 00:04:00,18,10.0
2000-01-01 00:06:00,26,18.0
2000-01-01 00:08:00,16,26.0


### LOffset
{'right', 'left'}, default None

This argument is also pretty self explanatory. Instead of changing any of the calculations, it just bumps the labels over by the specified amount of time.

In [162]:
minutes

2000-01-01 00:00:00     0
2000-01-01 00:01:00     2
2000-01-01 00:02:00     4
2000-01-01 00:03:00     6
2000-01-01 00:04:00     8
2000-01-01 00:05:00    10
2000-01-01 00:06:00    12
2000-01-01 00:07:00    14
2000-01-01 00:08:00    16
Freq: T, dtype: int64

In [183]:
df=pd.DataFrame()
df['no_offset'] = minutes.resample('2min').sum()
df['2min_offset'] = minutes.resample('2min',loffset='2T').sum()
df['4min_offset'] = minutes.resample('2min',loffset='4T').sum()
df

,no_offset,2min_offset,4min_offset
2000-01-01 00:00:00,2,NaN,NaN
2000-01-01 00:02:00,10,2.0,NaN
2000-01-01 00:04:00,18,10.0,2.0
2000-01-01 00:06:00,26,18.0,10.0
2000-01-01 00:08:00,16,26.0,18.0


### Base

Shifts the base time to calculate from by some time amount. As the documentation describes it, this function moves the 'origin'.

In [208]:
minutes.head().resample('30S').sum()

2000-01-01 00:00:00    0
2000-01-01 00:00:30    0
2000-01-01 00:01:00    2
2000-01-01 00:01:30    0
2000-01-01 00:02:00    4
2000-01-01 00:02:30    0
2000-01-01 00:03:00    6
2000-01-01 00:03:30    0
2000-01-01 00:04:00    8
Freq: 30S, dtype: int64

In [211]:
minutes.head().resample('30S',base=15).sum()

1999-12-31 23:59:45    0
2000-01-01 00:00:15    0
2000-01-01 00:00:45    2
2000-01-01 00:01:15    0
2000-01-01 00:01:45    4
2000-01-01 00:02:15    0
2000-01-01 00:02:45    6
2000-01-01 00:03:15    0
2000-01-01 00:03:45    8
Freq: 30S, dtype: int64

In [216]:
months.resample('15D5H',base=10.5).sum()

2000-01-16 05:30:00    0
2000-01-31 10:30:00    0
2000-02-15 15:30:00    2
2000-03-01 20:30:00    0
2000-03-17 01:30:00    4
2000-04-01 06:30:00    0
2000-04-16 11:30:00    6
2000-05-01 16:30:00    0
2000-05-16 21:30:00    8
Freq: 365H, dtype: int64